In [0]:
#all spark imports
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
#instantiate the spark session
spark = SparkSession.builder.appName("AssignmentTwo").getOrCreate()

1. Count the odd and even numbers usingfile‘integer.txt’download it from the Quercus. Show your code and output.

In [0]:
integerRDD = spark.sparkContext.textFile("/FileStore/tables/integer.txt", 4)
arrayRDD = integerRDD.map(lambda x: ( int(x)))
evenCount=arrayRDD.filter(lambda x: x%2 == 0).count()
print("EvenCount: %3d"% (evenCount))
oddcount=arrayRDD.filter(lambda x: x%2 != 0).count()
print("OddCount: %3d"% (oddcount))

EvenCount: 514
OddCount: 496


2 Calculate the salary sum per department using file ‘salary.txt’ download it from the Quercus. Show department name and salary sum. Show your code and output.

In [0]:
salaryRDD = spark.sparkContext.textFile("/FileStore/tables/salary.txt", 4)
arrayRDD = salaryRDD.map(lambda x: x.split(" "))
kvRDD = arrayRDD.map(lambda x: (x[0], float(x[1])))
sumRDD = kvRDD.reduceByKey(lambda x,y: x+y)
sumRDD.collect()

Out[4]: [('Developer', 3221394.0),
 ('Sales', 3488491.0),
 ('Research', 3328284.0),
 ('QA', 3360624.0),
 ('Marketing', 3158450.0)]

3 Implement MapReduce using Pyspark on file ‘shakespeare.txt’ download it from the Quercus. Show how many times these particular words appear in the document: Shakespeare, When, Lord, Library, GUTENBERG, WILLIAM, COLLEGE and WORLD. (Count exact words only)

In [0]:
import re 
shakespeareRDD = spark.sparkContext.textFile("/FileStore/tables/shakespeare_1-1.txt", 4)
counts=shakespeareRDD.flatMap(lambda line: line.split(" ")) \
                  .filter(lambda word: word == "When" or word=="Lord" or word == "Library" or word == "GUTENBERG" or word == "WILLIAM" or word == "COLLEGE" or word == "WORLD") \
                  .map(lambda word: (word, 1)) \
                  .reduceByKey(lambda a, b: a +b)

for element in counts.collect():
    print(element)

    


    



('WILLIAM', 115)
('WORLD', 98)
('When', 393)
('GUTENBERG', 99)
('COLLEGE', 98)
('Lord', 341)
('Library', 2)


4 Calculate top15 and bottom15 words using file‘shakespeare.txt’ download it from the Quercus. Show 15 words with most count and 15 words with least count. You can limit by 15 in ascending and descending order of count. Show your code and output.

In [0]:
shakespeareRDD = spark.sparkContext.textFile("/FileStore/tables/shakespeare_1-1.txt", 4)
counts=shakespeareRDD.flatMap(lambda line: line.split(" ")) \
                  .map(lambda word: (word, 1)) \
                  .reduceByKey(lambda a, b: a +b)\
                  .top(15,  key=lambda word: word[1])


for element in counts:
    print(element)

('', 231583)
('the', 11397)
('and', 8777)
('I', 8556)
('of', 7873)
('to', 7421)
('a', 5672)
('my', 4913)
('in', 4600)
('you', 4060)
('And', 3547)
('that', 3522)
('is', 3481)
('his', 3226)
('with', 3175)


In [0]:
shakespeareRDD = spark.sparkContext.textFile("/FileStore/tables/shakespeare_1-1.txt", 4)
counts=shakespeareRDD.flatMap(lambda line: line.split(" ")) \
                  .map(lambda word: (word, 1)) \
                  .reduceByKey(lambda a, b: a +b)\
                  .takeOrdered(15,  key=lambda word: word[1])


for element in counts:
    print(element)

('whatsoever.', 1)
('re-use', 1)
('eBook,', 1)
('file.', 1)
('Author:', 1)
('Posting', 1)
('1,', 1)
('Language:', 1)
('Character', 1)
('cooperation', 1)
('*This', 1)
('read!*', 1)
('SHAREWARE', 1)
('PUBLIC', 1)
('DOMAIN.', 1)


In [0]:
from pyspark.ml.feature import Imputer, StringIndexer, VectorAssembler
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator


1. Describe your data. Calculate top 12 movies with highest ratings and top 12 users who provided highest ratings. Show your code and output.


The data has 1501 rows and three columns. The columns in the data are movieId, ratings user give for the movie and userId
Data type of Columns are: 'movieId', 'int', 'rating', 'int', 'userId', 'int'.

In [0]:
path = "/FileStore/tables/movies.csv"
 
df = spark.read \
  .format("csv") \
  .option("inferSchema", True) \
  .option("header", True) \
  .option("path", path) \
  .load()

display(df)
# number of rows from the Dataframe
row = df.count()
# number of columns from the Dataframe
col = len(df.columns)

# data type of columns
datatype=df.dtypes
# printing
print(f'Dimension of the Dataframe is: {(row,col)}')
print(f'Number of Rows are: {row}')
print(f'Number of Columns are: {col}')
print(f'Data type of Columns are: {datatype}')


Dimension of the Dataframe is: (1501, 3)
Number of Rows are: 1501
Number of Columns are: 3
Data type of Columns are: [('movieId', 'int'), ('rating', 'int'), ('userId', 'int')]


movieId,rating,userId
2,3,0
3,1,0
5,2,0
9,4,0
11,1,0
12,2,0
15,1,0
17,1,0
19,1,0
21,1,0


In [0]:
df1=df.groupBy("movieId").sum('rating')

display(df1.sort("sum(rating)").tail(12))


df2=df.groupBy("userId").sum("rating")
display(df2.sort("sum(rating)").tail(12))

movieId,sum(rating)
62,36
18,36
88,37
23,37
49,39
51,40
2,40
22,41
68,41
90,45


userId,sum(rating)
18,90
28,91
8,93
9,95
2,95
24,98
12,102
14,102
26,108
23,111


2 Split dataset into train and test. Try 2 different combinations for e.g. (60/40, 70/30, 75/25 and 80/20). (Train your model and use collaborative filtering approach on 70 percent of your data and test with the other 30 percent and so on). Show your code and output.

In [0]:

# split data into training and test datasets 
train_df1, test_df1= df.randomSplit([0.7, 0.3], 1234)
train_df2, test_df2=df.randomSplit([0.8, 0.2], 1234)
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als1 = ALS(maxIter=5, regParam=0.01,userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")

model1 = als1.fit(train_df1)
model2=als1.fit(train_df2)





3  Explain MSE, RMSE and MAE. Compare and evaluate both of your models with evaluation metrics (RMSE or MAE), show your code and print your results. Describe which one works better and why?

All  three parameteres help evaluate  model accuracy

MSE (Mean Square Error): square difference between the orginal value and predicted value. The lesser the value of MSE the more close the predicted value is to the orginal value.
RMSE (Root Mean Square Error) which is the square root of MSE. the lesser the result of RMSE the more accurate the model is. The result of RMSE states that the orginal value is +/- RMSE  of the predicted value
MAE(Mean Absolute Error): It is the average difference between the orginal and pridicted value.

In [0]:
# Evaluate the model by computing the RMSE on the test data for model 0ne
predictions1 = model1.transform(test_df1)

evaluator1 = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse1 = evaluator1.evaluate(predictions1)
print("Root-mean-square error for model one (0.7,0.3) = " + str(rmse1))



# Evaluate the model by computing the RMSE on the test data for model Two
predictions2 = model2.transform(test_df2)

evaluator2 = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse2 = evaluator2.evaluate(predictions2)
print("Root-mean-square error for model two (0.8,0.2) = " + str(rmse2))

Root-mean-square error for model one (0.7,0.3) = 1.9847788424275474
Root-mean-square error for model two (0.8,0.2) = 1.6218664580955613


RMSE value for model one which is the data is split 70% training and 30% test data is approximatly 1.985 as shown above in the output. On the other hand RMSE for model two which is 80% training and 20% test data resulted 1.62 RMSE. The result suggest that the higher the proportion of the training dataset the more accurate the model is.

4 Now tune the parameters of your algorithm to get the best set of parameters. Explain different parameters of the algorithm which you have used for tuning your algorithm. Evaluate all your models again. Show your code with best values and output.

In [0]:
train_df, test_df= df.randomSplit([0.7, 0.3], 1234)
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")

#Tune model using ParamGridBuilder 
paramGrid = ParamGridBuilder()\
    .addGrid(als.regParam, [0.1, 0.01]) \
    .addGrid(als.rank, [10, 50])\
    .addGrid(als.maxIter, [1,3])\
    .build()
evaluators = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
# build cross validation
cv = CrossValidator(estimator =als, estimatorParamMaps = paramGrid,evaluator =evaluators, numFolds=3)
#Fit model to training data
model = cv.fit(train_df)
# Evaluate the model by computing the RMSE on the test data for the model
predictions = model.transform(test_df)
rmse = evaluators.evaluate(predictions)
#Extract best model from the cv model above
bestModel = model.bestModel
print("**Best Model**")
# Print "Rank"
print("  Rank:", bestModel._java_obj.parent().getRank())
# Print "MaxIter"
print("  MaxIter:", bestModel._java_obj.parent().getMaxIter())
# Print "RegParam"
print("  RegParam:", bestModel._java_obj.parent().getRegParam())
print("Root-mean-square error for the  model  = " + str(rmse))

/databricks/spark/python/pyspark/ml/util.py:839: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
  warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)
**Best Model**
  Rank: 50
  MaxIter: 3
  RegParam: 0.1
Root-mean-square error for the  model  = 1.0505386361339517


The different parameters used inorder to tune our model are : 
    regParam, [0.1, 0.01]) : which is a regularization. parametere default is 1.0 I have used set of 0.1 and 0.01 
    rank, [10, 50]: is the number of latent factor in the model default is 10. I have used set of 10,50 in the gridBuilder
    maxIter, [1,3]: is number of times your model iterates default to 10 and in the gridBuilder we have used set of 1 and 3

4  Calculate top 12 movies recommendations for userid 1 and userid 12. Show your code and output.

In [0]:
# a list of movies we are thinking to offer
user1_suggest = test_df.filter(train_df['userId']==1).select(['movieId', 'userId'])
user12_suggest = test_df.filter(train_df['userId']==12).select(['movieId', 'userId'])
# Top 12 movies recommndations for user one 
user1_recommendations= model.transform(user1_suggest)
display(user1_recommendations.orderBy('prediction').tail(12))


# Top 12 movies recommndations for user 12
user12_recommendations= model.transform(user12_suggest)
display(user12_recommendations.orderBy('prediction', decending=False).limit(12))

movieId,userId,prediction
6,1,1.2027411460876465
13,1,1.291556477546692
40,1,1.2950832843780518
93,1,1.3220633268356323
73,1,1.3668843507766724
9,1,1.4442086219787598
67,1,1.6449387073516846
56,1,1.7549599409103394
74,1,1.7646316289901733
63,1,1.8299444913864136


movieId,userId,prediction
31,12,0.65923315
78,12,0.69465554
14,12,0.76410866
86,12,0.7757331
21,12,0.8061414
15,12,0.84743696
4,12,1.0146977
24,12,1.1157546
82,12,1.2705686
51,12,1.2999612
